In [1]:
import requests
import random
import json
import tqdm

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'

DATA    = {
    "message"   : "How do you treat peach leaf curl?",
    "sender"    : str(random.randint(0, 100000))
}

r = requests.post(URL, json = DATA)

if r.status_code == 200:
    print(json.loads(r.text))
else:
    print('Service unavailable')

[{'recipient_id': '78084', 'text': 'Extracted slots</br>[Format: (<i>relation</i>) <strong>entity</strong> - <strong>value</strong>]:</br>Group 1:</br>(<i>pest</i>) <strong>name</strong> - <strong>peach leaf curl</strong></br>(<i>pest</i>) <strong>action</strong> - <strong>treat</strong></br></br>Composed 1 additional queries:</br>1) <i>peach leaf curl treat</i></br>'}, {'recipient_id': '78084', 'text': 'Results with slots improvement... Top 9 results.'}, {'recipient_id': '78084', 'custom': {'text': 'Here are my top results:', 'payload': 'collapsible', 'data': [{'title': '<p>1)<em>Peach Leaf Curl</em></br>(score: 0.89)</br>(source: <a href="http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html?src=exchbt" target="_blank">pestsIPM</a>)</p>', 'description': '<p><strong>Details</strong>: Peach leaf curl, also known as leaf curl, is a disease caused by the fungus Taphrina deformans. Peac</p></br><p><strong>Development</strong>: Peach leaf curl first appears in spring as reddish areas on developin

In [2]:
import random
import re

sender = random.randint(0, 100000)

# URL     = 'https://dev.api.chat.ask.eduworks.com/webhooks/rest/webhook'
URL     = 'http://localhost:5005/webhooks/rest/webhook'
DATA    = {
    "message"   : "", 
    "sender"    : str(sender)
}
def run_tests(questions):
    
    results = []
    for i, q in enumerate(tqdm.tqdm(questions)):

        DATA['message'] = q
        try:
            response = requests.post(URL, json = DATA)
            if response.status_code != 200:
                print(f'Error: Service at {URL} is unavailable, exit.')
                exit()

        except Exception as e:
            print(f'Error: Exception at posting question - "{q}", exit. {type(e).__name__}: "{e}".')
            exit()

        
        try:
            r = json.loads(response.text)
            success = False
            for link in r:
                if 'custom' in link:
                    success = True
                    r = link['custom']
                    break
        except Exception as e:
            print(f'Error: Failed on parsing response on question - "{q}", exit. {type(e).__name__}: "{e}".')
            exit()

        
        result = []
        if success:
           
            try:
                r = r['data']
                if len(r) == 0:
                    raise Exception
            except Exception as e:
                print(f'Error: Failed on parsing response on question - "{q}", exit. . {type(e).__name__}: "{e}".')

            for e in r:
                title   = re.findall("<em>(.*?)</em>"           , e['title'])[0]
                link    = re.findall("href=[\"\'](.*?)[\"\']"   , e['title'])[0]
                result.append((title, link))
            

            DATA['message'] = '/intent_affirm'
            try:
                response = requests.post(URL, json = DATA)
                if response.status_code != 200:
                    print(f'Error: Service at {URL} is unavailable, exit.')
                    exit()
            except Exception as e:
                print(f'Error: Exception at posting affirmative message on question - "{q}", exit. {type(e).__name__}: "{e}".')
                exit()
            
            try:
                r = json.loads(response.text)
                r = r[0]
                r = r['text']
                if 'Anything else I can help with?' != r:
                    raise Exception
            except Exception as e:
                print(f'Error: Failed on parsing response of affirmative message on question - "{q}", exit. {type(e).__name__}: "{e}".')
                exit()
        else:
            print(f'No results for question - "{q}"')
            result.append(('', ''))
            if (i+1)%5 == 0:
                print(f'Finished {i+1} questions...')
        
        results.append(result)
    
    print(f'Finished querying all questions for scoring')

    return results


In [3]:
import pandas as pd

FINAL_DATA = './data/transformed/result.pkl'

df = pd.read_pickle(FINAL_DATA)

df.sample(10)

,Question,ExpectedAnswer,URL,Source
3,What is the best way to get rid of peach leaf ...,[Pest Notes: Peach Leaf Curl],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html],UCIPM
38,There are a lot of flies on my persimmon tree....,NaN,[http://ipm.ucanr.edu/PMG/GARDEN/FRUIT/PESTS/f...,AE
3,I started noticing some sort of bug that made ...,NaN,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7420.htm...,AE
43,How do you keep mosquitoes away?,[Pest Notes: Mosquitoes],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7451.html],UCIPM
6,Is there treatment for ash borers on ash trees...,NaN,[http://ipm.ucanr.edu/PMG/GARDEN/PLANTS/INVERT...,AE
15,A white powder-like substance is appearing at ...,NaN,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn74174.html],AE
47,I have a garden full of weeds. How do I clear ...,[Pest Notes: Weed Management in landscapes],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7441.html],UCIPM
1,How can I get rid of peach leaf curl?,[Pest Notes: Peach Leaf Curl],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7426.html],UCIPM
26,I found bugs in my cabinets where I store food...,NaN,[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7452.htm...,AE
35,How do you get rid of fungus gnats?,[Pest Notes: Fungus Gnats],[http://ipm.ucanr.edu/PMG/PESTNOTES/pn7448.html],UCIPM


In [4]:
answers     = df['URL'].values.tolist()
results     = run_tests(df['Question'])

100%|██████████| 97/97 [02:08<00:00,  1.32s/it]

Finished querying all questions for scoring


In [7]:
scores = []
for i, r in enumerate(results):
    answer = answers[i]
    topn = [False, False, False, False]
    for i1, r1 in enumerate(r):
        if r1[1].split('?')[0] in answer:
            if i1 == 0:
                topn[0] = True
            if i1 < 3:
                topn[1] = True
            if i1 < 5:
                topn[2] = True
            topn[3] = True

    scores.append(topn)
    

In [8]:
for i, score in enumerate(scores):
    if not any(score):
        print(f'------------------------------------------------------------------')
        print(f'Question number : {i+1}')
        print(f'Question        : {df["Question"].iloc[i]}')
        print(f'The answers     : {answers[i]}')

------------------------------------------------------------------
Question number : 5
Question        : What are the big black bugs on my patio at night?
The answers     : ['http://ipm.ucanr.edu/PMG/PESTNOTES/pn7467.html']
------------------------------------------------------------------
Question number : 13
Question        : What does rust on roses look like?
The answers     : ['http://ipm.ucanr.edu/PMG/PESTNOTES/pn7463.html']
------------------------------------------------------------------
Question number : 14
Question        : My rose leaves are covered with dusty orange spots. Is it rust? What can I do about it?
The answers     : ['http://ipm.ucanr.edu/PMG/PESTNOTES/pn7463.html']
------------------------------------------------------------------
Question number : 21
Question        : Why is my tree dying?
The answers     : ['http://ipm.ucanr.edu/PMG/PESTNOTES/pn74109.html', 'http://ipm.ucanr.edu/PMG/PESTNOTES/pn74171.html']
------------------------------------------------------

In [6]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 97 results, following correct:
Top 1 : 27  (27.84%)
Top 3 : 30  (30.93%)
Top 5 : 35  (36.08%)
Top 10: 40  (41.24%)


In [6]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 97 results, following correct:
Top 1 : 27  (27.84%)
Top 3 : 30  (30.93%)
Top 5 : 34  (35.05%)
Top 10: 39  (40.21%)


In [12]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
for topn in scores:
    if topn[0]: top1    += 1
    if topn[1]: top3    += 1
    if topn[2]: top5    += 1
    if topn[3]: top10   += 1

print(f'Out of {len(results)} results, following correct:'  )
print(f'Top 1 : {top1 :<3d} ({top1 /len(results) * 100:<.2f}%)')
print(f'Top 3 : {top3 :<3d} ({top3 /len(results) * 100:<.2f}%)')
print(f'Top 5 : {top5 :<3d} ({top5 /len(results) * 100:<.2f}%)')
print(f'Top 10: {top10:<3d} ({top10/len(results) * 100:<.2f}%)')

Out of 97 results, following correct:
Top 1 : 28  (28.87%)
Top 3 : 35  (36.08%)
Top 5 : 38  (39.18%)
Top 10: 43  (44.33%)
